## Underwater stuff 

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import copy 
import cv2

from tqdm import tqdm
from pathlib import Path, PosixPath
from PIL import Image, ImageDraw
from pydantic import BaseModel
from typing import Optional 


import matplotlib.pyplot as plt
plt.style.use("bmh")
%matplotlib inline

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

This tutorial is used to understand the data. 
## List of topics. 
- [x] visualize the images.  
- [x] visualize the image with annot boxes.  
- [x] understand video_id and create a video and visualize them.  
- [x] video's with bboxes (vis and data)  
- [ ] bbox stats  

In [ ]:
root = Path("/kaggle/input/tensorflow-great-barrier-reef/")
root

In [ ]:
list(root.iterdir())

In [ ]:
df = pd.read_csv(root/"train.csv")
df.head()

In [ ]:
total_images = {i.name:len(list(i.glob("*.jpg"))) for i in list((root / "train_images").glob("*"))}
print(total_images)
df["video_id"].value_counts()

In [ ]:
class Annot(BaseModel):
    bbox: np.ndarray
    dtype: str
    
    class Config:
        arbitrary_types_allowed = True        
        

class ImageStore(BaseModel):
    img_loc: PosixPath
    video_id: str
    frame_id: str 
    annot: Annot
    img: np.ndarray
    vis_img: Optional[np.ndarray]
    
    class Config:
        arbitrary_types_allowed = True

In [ ]:
class GBR:
    def __init__(self, root, df_loc, only_annots=True):
        self.df = pd.read_csv(df_loc) if isinstance(df_loc, (str, PosixPath)) else df_loc
        self.root = root 
        self.df = self.df[self.df["annotations"] != "[]"].reset_index(drop=True) if only_annots else self.df 
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        meta = self.df.iloc[idx]
        video_id = f"video_{meta['video_id']}"
        loc = self.root / "train_images" / video_id / (str(meta["video_frame"])+".jpg")
        if loc.exists():
            img = np.asarray(Image.open(loc))
            annot = Annot(bbox=np.asarray([[i["x"], i["y"], i["width"], i["height"]] for i in eval(meta["annotations"]) if i != "[]"]), dtype="xywh")
            data = ImageStore(img= img, annot=annot, img_loc=loc, video_id=loc.parent.name, frame_id=loc.name)
            return data
        else:
            print("Image not present")
    
    @staticmethod
    def draw_bboxes(img, bboxes, line_width=5, color=(255, 0, 0)):
        img = Image.fromarray(copy.deepcopy(img))
        draw = ImageDraw.Draw(img)
        lw = line_width or max(round(sum(img.shape) / 2 * 0.003), 2)
        for annot in bboxes:
            x, y, w, h = annot
            draw.rectangle([x, y, x+w, y+h], width=lw, outline="red")
        return img
    
    def vis_random(self):
        idx = np.random.randint(len(self))
        data = self[idx]
        if data is not None:
            vis_img = self.draw_bboxes(data.img, data.annot.bbox) if len(data.annot.bbox) > 0 else data.img 
            print(f"visualizing: video_id: {data.video_id}, frame: {data.frame_id}")
            fig, ax = plt.subplots(figsize=(12, 7.5), nrows=1, ncols=1)
            ax.imshow(vis_img)
            ax.axis("off")

In [ ]:
gbr = GBR(root, df, only_annots=True)

In [ ]:
## visualizing a random frame. Can u recognize a object (starfish) just looking at the frame. 
gbr.vis_random()

## How many videos are present in the dataset ?
- `sequence` column gives individual video snippets unique_ids. 
- `sequence_frame` gives the frame number within the video. 
- The below plot also tells where star fishes are located within the video along with their count. 

In [ ]:
df["contains_star_fish"] = df["annotations"].apply(lambda x: len(eval(x)))
df.head()

In [ ]:
fps = 30
time_per_sequence = (df.groupby(["video_id", "sequence"])["video_frame"].count()/fps).reset_index()
time_per_sequence.columns = ["video_id", "sequence", "time (sec)"]
time_per_sequence.head()

In [ ]:
time_per_sequence["time (sec)"].hist(figsize=(8, 3.5), bins=20)
plt.title("time (sec)")
plt.show()

- Visualizing the appearance of star fish within a video_sequence. 
- Colors represent the count of star fish within the video. On x-axis we have where the star fish is see within the `sequence`.
- y_axis we have the video_frame number. From this we can make that a single video is stripped into several small video chunks. 

In [ ]:
fig, ax = plt.subplots(figsize=(8*3, 3.5), nrows=1, ncols=3)
for i in range(df["video_id"].unique().shape[0]):
    df_ = df[df["video_id"] == i]
    ax.flat[i].grid(False)
    pcm = ax.flat[i].scatter(df_["sequence_frame"].values, df_["video_frame"].values, c=df_["contains_star_fish"].values, cmap='RdBu_r')
    ax.flat[i].set_title(f"video_id: {i}")
    #ax.flat[i].legend(loc="upper right")
fig.colorbar(pcm, shrink=1)
plt.show()

## Convert frame to videos 

In [ ]:
def frames2video_gbr(df, save_folder, vis_bbox=False, fps:int=30):
    ## convert a sequence to video using video_id and sequence as folder name. 
    video_id = df["video_id"].unique()[0]
    out = None 
    d_ = GBR(root, df_, only_annots=False)
    for i in tqdm(range(len(d_))):
        data = d_[i]
        if data is None:
            continue
        height, width, layers = data.img.shape
        size = (width, height)
        if out is None:
            out = cv2.VideoWriter((save_folder/f"video_{video_id}-{sequence}.mp4").as_posix(), cv2.VideoWriter_fourcc(*'MP4V'), fps, size)
        
        if vis_bbox:
            img = GBR.draw_bboxes(data.img, data.annot.bbox) if len(data.annot.bbox) > 0 else data.img 
            img = img if isinstance(img, np.ndarray) else np.asarray(img)
        else:
            img = data.img
        
        out.write(img[:, :, ::-1])
    out.release()

## Save a video
- one raw video 
- same video with bboxes on the image.

In [ ]:
save_path=Path("/kaggle/working/") / "raw"
save_path_bbox=Path("/kaggle/working/") / "bbox"
save_path.mkdir(exist_ok=True)
save_path_bbox.mkdir(exist_ok=True)
fps = 24
## select a video 
sequence = df["sequence"].unique()[3]
df_ = df[df["sequence"] == sequence].reset_index(drop=True)
print(df_.shape)
frames2video_gbr(df_, save_path, vis_bbox=False, fps=fps)
frames2video_gbr(df_, save_path_bbox, vis_bbox=True, fps=fps)

In [ ]:
# from IPython.display import Video
# Video(list(Path("/kaggle/working/raw").glob("*"))[0].as_posix())